In [1]:
import logging
import os
import json
import base64
from io import BytesIO
from PIL import Image
import subprocess
import time
from collections import namedtuple
from pathlib import Path

import google.cloud.aiplatform as aiplatform
from google.cloud import storage

logging.getLogger().setLevel(logging.INFO)

In [2]:
APP_NAME = 'ViT-model'
MODEL_PT_FILEPATH = 'saved_models/VisionTransformers'
MAR_MODEL_OUT_PATH = 'serve'
handler = 'predictor/handler.py'
MODEL_DISPLAY_NAME = 'ViT-model'
model_version = 1
PROJECT_ID = 'alberto-playground'
BUCKET_NAME = 'alberto-vit-playground'
CUSTOM_PREDICTOR_IMAGE_URI = f"gcr.io/{PROJECT_ID}/pytorch_predict_vit"

In [3]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

In [4]:
endpoint_display_name = f"{APP_NAME}-endpoint"
filter = f'display_name="{endpoint_display_name}"'

for endpoint_info in aiplatform.Endpoint.list(filter=filter):
    print(
        f"Endpoint display name = {endpoint_info.display_name} resource id ={endpoint_info.resource_name} "
    )

endpoint = aiplatform.Endpoint(endpoint_info.resource_name)

Endpoint display name = ViT-model-endpoint resource id =projects/634066980332/locations/us-central1/endpoints/8755274752538443776 


In [5]:
endpoint.list_models()

[id: "945036841143238656"
 model: "projects/634066980332/locations/us-central1/models/5377220989266427904"
 display_name: "ViT-model-v1"
 create_time {
   seconds: 1690648838
   nanos: 485629000
 }
 dedicated_resources {
   machine_spec {
     machine_type: "n1-standard-4"
   }
   min_replica_count: 1
   max_replica_count: 1
 }
 model_version_id: "1"]

In [6]:
def limit_img_size(img, target_filesize, tolerance=5):
    # img = img_orig = Image.open(img_filename)
    aspect = img.size[0] / img.size[1]

    while True:
        with BytesIO() as buffer:
            img.save(buffer, format="JPEG")
            data = buffer.getvalue()
        filesize = len(data)    
        size_deviation = filesize / target_filesize
        print("size: {}; factor: {:.3f}".format(filesize, size_deviation))

        if size_deviation <= (100 + tolerance) / 100:
            # filesize fits
            return data
        else:
            # filesize not good enough => adapt width and height
            # use sqrt of deviation since applied both in width and height
            new_width = img.size[0] / size_deviation**0.5    
            new_height = new_width / aspect
            # resize from img_orig to not lose quality
            img = img.resize((int(new_width), int(new_height)))

In [8]:
bucket = storage.Client().bucket(BUCKET_NAME)
blob = bucket.get_blob('samples/pexels-helena-lopes-1996332.jpg')    
img = Image.open(BytesIO(blob.download_as_bytes()))
image_bytes = limit_img_size(img,  15000,tolerance = 5)
encoded_string = base64.b64encode(image_bytes)

size: 281851; factor: 18.790
size: 17194; factor: 1.146
size: 14442; factor: 0.963


In [9]:
test_instance = [{"body": 
    {
    "file": {
        "filename": "pexels-helena-lopes-1996332.jpg",
        "content": f"{str(encoded_string.decode('utf-8'))}" }
    }
}
]

In [12]:
prediction = endpoint.predict(instances=test_instance)

In [13]:
prediction

Prediction(predictions=[{'probabilities': {'airplane': 0.1399828493595123, 'ship': 0.1539709866046906, 'truck': 0.02285150066018105, 'cat': 0.06561080366373062, 'dog': 0.1819493472576141, 'automobile': 0.07853943854570389, 'deer': 0.06153243407607079, 'frog': 0.1282880455255508, 'horse': 0.09804531931877136, 'bird': 0.06922928243875504}, 'response': 'dog'}], deployed_model_id='945036841143238656', model_version_id='1', model_resource_name='projects/634066980332/locations/us-central1/models/5377220989266427904', explanations=None)